In [1]:
from numpy.core.numeric import True_
import sys
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.linear_model import LogisticRegression
#from matplotlib import pyplot as plt
#from PIL import Image, ImageDraw
#from IPython import display
#from torchvision import transforms

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [3]:
data = 'german_credit'
df = pd.read_csv('../datasets/'+data+'_sc.csv')

In [4]:
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,0.508000,0.029412,0.828912,0.778292,0.050567,0.824457,0.746850,1.000000,0.733515,0.70011,...,0.786926,0.857143,0.724785,0.739076,0.333333,0.704754,0.0,0.720247,0.692635,1
1,0.610000,0.647059,0.681168,0.778292,0.313690,0.640232,0.693235,0.333333,0.648553,0.70011,...,0.786926,0.053571,0.724785,0.739076,0.000000,0.704754,0.0,0.686265,0.692635,0
2,0.882785,0.117647,0.828912,0.562745,0.101574,0.640232,0.775429,0.333333,0.733515,0.70011,...,0.786926,0.535714,0.724785,0.739076,0.000000,0.719900,1.0,0.686265,0.692635,1
3,0.508000,0.558824,0.681168,0.679670,0.419941,0.640232,0.775429,0.333333,0.733515,0.80566,...,0.693991,0.464286,0.724785,0.593578,0.000000,0.704754,1.0,0.686265,0.692635,1
4,0.508000,0.294118,0.682022,0.620000,0.254209,0.640232,0.693235,0.666667,0.733515,0.70011,...,0.565806,0.607143,0.724785,0.593578,0.333333,0.704754,1.0,0.686265,0.692635,0


In [5]:
target = 'class'
y = df[target].values
del df[target]
x = df.values

In [6]:
# train the classifier to be explained -> Logistic Regression Model (can be replaced with a MLP)
# clf = LogisticRegression(max_iter=1000, fit_intercept=False, class_weight='balanced')
# clf.fit(x, y)

In [7]:
# Convert numpy arrays to PyTorch tensors
x_train_tensor = torch.FloatTensor(x).to(device)
y_train_tensor = torch.LongTensor(y).to(device)

In [8]:
# Create a TensorDataset
dataset = TensorDataset(x_train_tensor, y_train_tensor)

In [9]:
# Create a DataLoader
dl_batch_size = 32  # Set your desired batch size
train_dataloader = DataLoader(dataset, batch_size=dl_batch_size, shuffle=True)

In [10]:
num_epochs = 500

In [11]:
x.shape[1]

20

In [12]:
latent_dims = 8
# capacity = no. of hidden neurons in a layer
capacity = 8
variational_beta = 1

In [13]:
latent_dims

8

In [14]:
class Encoder(nn.Module):
    def __init__(self, input_dim):
        super(Encoder, self).__init__()
        c = capacity
        self.fc1 = nn.Linear(in_features=input_dim, out_features=c*2)
        self.fc2 = nn.Linear(in_features=c*2, out_features=latent_dims)
        self.fc3 = nn.Linear(in_features=c*2, out_features=latent_dims)
            
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x_mu = self.fc2(x)
        x_logvar = self.fc3(x)
        return x_mu, x_logvar

class Decoder(nn.Module):
    def __init__(self, output_dim):
        super(Decoder, self).__init__()
        c = capacity
        self.fc = nn.Linear(in_features=latent_dims, out_features=c*2)
        self.fc_output = nn.Linear(in_features=c*2, out_features=output_dim)
            
    def forward(self, x):
        x = F.relu(self.fc(x))
        x_recon = self.fc_output(x)
        return x_recon

class VariationalAutoencoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = Encoder(input_dim)
        self.decoder = Decoder(output_dim)
    
    def forward(self, x):
        latent_mu, latent_logvar = self.encoder(x)
        latent = self.latent_sample(latent_mu, latent_logvar)
        x_recon = self.decoder(latent)
        return x_recon, latent_mu, latent_logvar
    
    def latent_sample(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')  # Mean Squared Error loss for tabular data
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + variational_beta * kldivergence

In [15]:
# Train VAE
print(device)
vae = VariationalAutoencoder(input_dim=x.shape[1], output_dim=x.shape[1])
vae = vae.to(device)

optimizer = torch.optim.Adam(params=vae.parameters(), lr=1e-3, weight_decay=1e-5)

train_loss_avg = []

train_vae = True

if train_vae == False:   
    filename = 'blood_alcohol_vae.pth'
    vae.load_state_dict(torch.load(filename))
    vae.eval()
    print('done')

else:
    for epoch in range(num_epochs):
        vae.train()
        train_loss_avg.append(0)
        num_batches = 0

        for x_batch, _ in train_dataloader:

            x_batch = x_batch.to(device)

            # Forward pass
            x_recon, latent_mu, latent_logvar = vae(x_batch)
            loss = vae_loss(x_recon, x_batch, latent_mu, latent_logvar)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_avg[-1] += loss.item()
            num_batches += 1

        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average loss: %f' % (epoch + 1, num_epochs, train_loss_avg[-1]))

    # Save the model
    model_name = data+'_vae.pth'
    torch.save(vae.state_dict(), model_name)

mps
Epoch [1 / 500] average loss: 266.498453
Epoch [2 / 500] average loss: 170.072534
Epoch [3 / 500] average loss: 113.160004
Epoch [4 / 500] average loss: 82.119071
Epoch [5 / 500] average loss: 64.672240
Epoch [6 / 500] average loss: 55.196171
Epoch [7 / 500] average loss: 46.317583
Epoch [8 / 500] average loss: 39.425421
Epoch [9 / 500] average loss: 34.083787
Epoch [10 / 500] average loss: 30.278348
Epoch [11 / 500] average loss: 28.589521
Epoch [12 / 500] average loss: 27.350516
Epoch [13 / 500] average loss: 25.679847
Epoch [14 / 500] average loss: 24.943194
Epoch [15 / 500] average loss: 24.480754
Epoch [16 / 500] average loss: 23.314376
Epoch [17 / 500] average loss: 23.186511
Epoch [18 / 500] average loss: 22.433914
Epoch [19 / 500] average loss: 22.041854
Epoch [20 / 500] average loss: 21.414090
Epoch [21 / 500] average loss: 21.413179
Epoch [22 / 500] average loss: 21.259710
Epoch [23 / 500] average loss: 20.875183
Epoch [24 / 500] average loss: 20.711287
Epoch [25 / 500] a

Epoch [199 / 500] average loss: 19.205082
Epoch [200 / 500] average loss: 19.205043
Epoch [201 / 500] average loss: 19.168631
Epoch [202 / 500] average loss: 19.242912
Epoch [203 / 500] average loss: 19.196494
Epoch [204 / 500] average loss: 19.217711
Epoch [205 / 500] average loss: 19.164024
Epoch [206 / 500] average loss: 19.197054
Epoch [207 / 500] average loss: 19.211779
Epoch [208 / 500] average loss: 19.215070
Epoch [209 / 500] average loss: 19.188533
Epoch [210 / 500] average loss: 19.177548
Epoch [211 / 500] average loss: 19.211694
Epoch [212 / 500] average loss: 19.213663
Epoch [213 / 500] average loss: 19.184469
Epoch [214 / 500] average loss: 19.216960
Epoch [215 / 500] average loss: 19.188162
Epoch [216 / 500] average loss: 19.244334
Epoch [217 / 500] average loss: 19.234200
Epoch [218 / 500] average loss: 19.189973
Epoch [219 / 500] average loss: 19.179941
Epoch [220 / 500] average loss: 19.209091
Epoch [221 / 500] average loss: 19.200717
Epoch [222 / 500] average loss: 19

Epoch [395 / 500] average loss: 19.177728
Epoch [396 / 500] average loss: 19.212286
Epoch [397 / 500] average loss: 19.192533
Epoch [398 / 500] average loss: 19.218516
Epoch [399 / 500] average loss: 19.216990
Epoch [400 / 500] average loss: 19.174568
Epoch [401 / 500] average loss: 19.198813
Epoch [402 / 500] average loss: 19.188350
Epoch [403 / 500] average loss: 19.226080
Epoch [404 / 500] average loss: 19.199708
Epoch [405 / 500] average loss: 19.195521
Epoch [406 / 500] average loss: 19.187538
Epoch [407 / 500] average loss: 19.201079
Epoch [408 / 500] average loss: 19.195918
Epoch [409 / 500] average loss: 19.177217
Epoch [410 / 500] average loss: 19.200938
Epoch [411 / 500] average loss: 19.211450
Epoch [412 / 500] average loss: 19.179285
Epoch [413 / 500] average loss: 19.185565
Epoch [414 / 500] average loss: 19.194523
Epoch [415 / 500] average loss: 19.182304
Epoch [416 / 500] average loss: 19.213813
Epoch [417 / 500] average loss: 19.187969
Epoch [418 / 500] average loss: 19

In [16]:
c2c_latent_dims = latent_dims-1
print(c2c_latent_dims)
c2c_capacity = latent_dims
print(c2c_capacity)
c2c_variational_beta = 1
label_size = 2  # Assuming two classes

7
8


In [17]:
class c2c_Encoder(nn.Module):
    def __init__(self):
        super(c2c_Encoder, self).__init__()
        self.fc1 = nn.Linear(in_features=latent_dims + label_size, out_features=c2c_capacity)
        self.fc_mu = nn.Linear(in_features=c2c_capacity, out_features=c2c_latent_dims)
        self.fc_logvar = nn.Linear(in_features=c2c_capacity, out_features=c2c_latent_dims)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x_mu = self.fc_mu(x)
        x_logvar = self.fc_logvar(x)
        return x_mu, x_logvar

class c2c_Decoder(nn.Module):
    def __init__(self):
        super(c2c_Decoder, self).__init__()
        self.fc2 = nn.Linear(in_features=c2c_latent_dims + label_size, out_features=c2c_capacity)
        self.fc1 = nn.Linear(in_features=c2c_capacity, out_features=latent_dims)
            
    def forward(self, x):
        x = self.fc2(x)
        x = self.fc1(x)
        return x
    
class c2c_VariationalAutoencoder(nn.Module):
    def __init__(self):
        super(c2c_VariationalAutoencoder, self).__init__()
        self.encoder = c2c_Encoder()
        self.decoder = c2c_Decoder()
    
    def forward(self, x):
        x,label = torch.split(x,(latent_dims,label_size),dim = 1)
        c2c_latent_mu, c2c_latent_logvar = self.encoder(torch.cat((x,label), dim=1))
        c2c_latent = self.latent_sample(c2c_latent_mu, c2c_latent_logvar)
        x_recon = self.decoder(torch.cat((c2c_latent,label), dim=1))
        return x_recon, c2c_latent_mu, c2c_latent_logvar
    
    def latent_sample(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
def c2c_vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x.view(-1, latent_dims), x.view(-1, latent_dims), reduction='sum')
    
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + c2c_variational_beta * kldivergence

In [18]:
print(device)
c2c_vae = c2c_VariationalAutoencoder()
c2c_vae = c2c_vae.to(device)

optimizer = torch.optim.Adam(params=c2c_vae.parameters(), lr=1e-3, weight_decay=1e-5)

train_loss_avg = []

train_c2c_vae = True

if train_c2c_vae == False:
    filename = 'blood_alcohol_c2c.pth'
    c2c_vae.load_state_dict(torch.load(filename))
    c2c_vae.eval()
    print('done')
else:
    for epoch in range(num_epochs):
        c2c_vae.train()
        train_loss_avg.append(0)
        num_batches = 0

        for i in range(dl_batch_size):

            x_batch1, labels1 = next(iter(train_dataloader))
            x_batch2, labels2 = next(iter(train_dataloader))

            x_batch1 = x_batch1.to(device)
            x_batch_recon1, latent_mu1, latent_logvar1 = vae(x_batch1)

            x_batch2 = x_batch2.to(device)
            x_batch_recon2, latent_mu2, latent_logvar2 = vae(x_batch2) 

            latent_diff = latent_mu1 - latent_mu2 
            label_diff = torch.cat((labels1.unsqueeze(1), labels2.unsqueeze(1)), dim=1)
            #print(label_diff)

            latent_diff = latent_diff.to(device)
            label_diff = label_diff.float().to(device)

            c2c_input = torch.cat((latent_diff,label_diff), dim=1)
            c2c_input = c2c_input.to(device)

            c2c_recon, c2c_latent_mu, c2c_latent_logvar = c2c_vae(c2c_input)
            loss = c2c_vae_loss(c2c_recon, latent_diff.detach(), c2c_latent_mu, c2c_latent_logvar)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            train_loss_avg[-1] += loss.item()
            num_batches += 1

        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average reconstruction error: %f' % (epoch + 1, num_epochs, train_loss_avg[-1]))

    # Save the model
    model_name = data+'_c2c.pth'
    torch.save(c2c_vae.state_dict(), model_name)

mps
Epoch [1 / 500] average reconstruction error: 29.933866
Epoch [2 / 500] average reconstruction error: 14.398626
Epoch [3 / 500] average reconstruction error: 7.098935
Epoch [4 / 500] average reconstruction error: 3.545524
Epoch [5 / 500] average reconstruction error: 1.890031
Epoch [6 / 500] average reconstruction error: 1.250830
Epoch [7 / 500] average reconstruction error: 0.840457
Epoch [8 / 500] average reconstruction error: 0.598441
Epoch [9 / 500] average reconstruction error: 0.461791
Epoch [10 / 500] average reconstruction error: 0.356083
Epoch [11 / 500] average reconstruction error: 0.279868
Epoch [12 / 500] average reconstruction error: 0.232992
Epoch [13 / 500] average reconstruction error: 0.173924
Epoch [14 / 500] average reconstruction error: 0.146099
Epoch [15 / 500] average reconstruction error: 0.120342
Epoch [16 / 500] average reconstruction error: 0.102642
Epoch [17 / 500] average reconstruction error: 0.084239
Epoch [18 / 500] average reconstruction error: 0.07

Epoch [147 / 500] average reconstruction error: 0.008101
Epoch [148 / 500] average reconstruction error: 0.008130
Epoch [149 / 500] average reconstruction error: 0.008188
Epoch [150 / 500] average reconstruction error: 0.008451
Epoch [151 / 500] average reconstruction error: 0.008091
Epoch [152 / 500] average reconstruction error: 0.008026
Epoch [153 / 500] average reconstruction error: 0.008012
Epoch [154 / 500] average reconstruction error: 0.008126
Epoch [155 / 500] average reconstruction error: 0.008365
Epoch [156 / 500] average reconstruction error: 0.008214
Epoch [157 / 500] average reconstruction error: 0.008410
Epoch [158 / 500] average reconstruction error: 0.008122
Epoch [159 / 500] average reconstruction error: 0.007864
Epoch [160 / 500] average reconstruction error: 0.008026
Epoch [161 / 500] average reconstruction error: 0.008514
Epoch [162 / 500] average reconstruction error: 0.008224
Epoch [163 / 500] average reconstruction error: 0.008286
Epoch [164 / 500] average recon

Epoch [291 / 500] average reconstruction error: 0.008421
Epoch [292 / 500] average reconstruction error: 0.008495
Epoch [293 / 500] average reconstruction error: 0.008616
Epoch [294 / 500] average reconstruction error: 0.008391
Epoch [295 / 500] average reconstruction error: 0.008324
Epoch [296 / 500] average reconstruction error: 0.008881
Epoch [297 / 500] average reconstruction error: 0.008500
Epoch [298 / 500] average reconstruction error: 0.008348
Epoch [299 / 500] average reconstruction error: 0.008243
Epoch [300 / 500] average reconstruction error: 0.008304
Epoch [301 / 500] average reconstruction error: 0.008706
Epoch [302 / 500] average reconstruction error: 0.008683
Epoch [303 / 500] average reconstruction error: 0.007749
Epoch [304 / 500] average reconstruction error: 0.008708
Epoch [305 / 500] average reconstruction error: 0.008192
Epoch [306 / 500] average reconstruction error: 0.008755
Epoch [307 / 500] average reconstruction error: 0.008917
Epoch [308 / 500] average recon

Epoch [435 / 500] average reconstruction error: 0.008452
Epoch [436 / 500] average reconstruction error: 0.008962
Epoch [437 / 500] average reconstruction error: 0.008167
Epoch [438 / 500] average reconstruction error: 0.008557
Epoch [439 / 500] average reconstruction error: 0.008722
Epoch [440 / 500] average reconstruction error: 0.008424
Epoch [441 / 500] average reconstruction error: 0.008643
Epoch [442 / 500] average reconstruction error: 0.009011
Epoch [443 / 500] average reconstruction error: 0.008141
Epoch [444 / 500] average reconstruction error: 0.008464
Epoch [445 / 500] average reconstruction error: 0.007997
Epoch [446 / 500] average reconstruction error: 0.008227
Epoch [447 / 500] average reconstruction error: 0.008378
Epoch [448 / 500] average reconstruction error: 0.008480
Epoch [449 / 500] average reconstruction error: 0.008136
Epoch [450 / 500] average reconstruction error: 0.008809
Epoch [451 / 500] average reconstruction error: 0.008777
Epoch [452 / 500] average recon